In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [3]:
# Load all data (except unknown/manual testing)
# Original Desktop Runs+Re-Runs were performed between 2024-02-05 and 2024-02-12
# Additional browsers + more than one response_id per parsing URL was run between 2024-02-16 and TBD
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown'; -- AND "Result".created_at < '2024-02-19';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [4]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)

In [5]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [6]:
# Both basic + parsing mode
# Each test should have results 5+ times (such that we can better reason about noise + majority voting makes sense)
# Basic was executed 20x+ on Linux, 5x on Mac
# Parsing was executed 5+ on Linux, 2x on Mac (currently executing a third time)
# (In addition, a couple of "repeat"-mode runs were performed in the beginning
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"])["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,selenium,42,chrome,121,Ubuntu 22.04,headless-new,2179756
1,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,2107288
2,selenium,43,firefox,122,Ubuntu 22.04,headless,2105082
3,selenium,44,edge,121,Ubuntu 22.04,headless-new,2102158
4,selenium,48,chrome,120,Ubuntu 22.04,headless-new,1769920
5,selenium,47,chrome,122,Ubuntu 22.04,headless-new,1763360
6,selenium,49,firefox,121,Ubuntu 22.04,headless,1751629
7,selenium,46,safari,17.3,macOS 14.3,real,417257


# Error + Timeout Analysis
- Only timeout exist, no other systematic failures anymore
- Systematic timeout: TAO in Safari -> fixed
- Couple of random timeouts in other tests
  - More basic tests than parsing tests (even though less exist)
  - Code 302 more often than others
  - referrer-iframe, oac_*, framing_iframe most often

In [7]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                            test_status         
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... 0            2107269
                                                   2                 19
chrome Ubuntu 22.04 120 selenium headless-new      0            1769904
                                                   2                 16
chrome Ubuntu 22.04 121 selenium headless-new      0            2179745
                                                   2                 11
chrome Ubuntu 22.04 122 selenium headless-new      0            1763343
                                                   2                 17
edge Ubuntu 22.04 121 selenium headless-new        0            2102050
                                                   2                108
firefox Ubuntu 22.04 121 selenium headless         0            1751581
                                                   2                 48
firefox Ubuntu 22.04 122 selenium headless         0            2105022
                                                   2                 60
safari macOS 14.3 17.3 selenium real               0             416992
                                                   2                265

In [8]:
# Teststatus == 2 (timeout)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img/TAO (only Safari, fixed!)
# Others quite rare
display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
safari macOS 14.3 17.3 selenium real,265
edge Ubuntu 22.04 121 selenium headless-new,108
firefox Ubuntu 22.04 122 selenium headless,60
firefox Ubuntu 22.04 121 selenium headless,48
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,19
chrome Ubuntu 22.04 122 selenium headless-new,17
chrome Ubuntu 22.04 120 selenium headless-new,16
chrome Ubuntu 22.04 121 selenium headless-new,11


count
resp_type status_code test_name             browser                                                  
basic     200         referrer_iframe       safari macOS 14.3 17.3 selenium real                  118
          302         referrer_iframe       safari macOS 14.3 17.3 selenium real                   74
parsing   200         perfAPI_img           safari macOS 14.3 17.3 selenium real                   32
                      fullscreen_iframe     edge Ubuntu 22.04 121 selenium headless-new            31
                      framing_iframe        firefox Ubuntu 22.04 122 selenium headless             29
basic     200         oac_iframe            safari macOS 14.3 17.3 selenium real                   24
          302         oac_iframe            edge Ubuntu 22.04 121 selenium headless-new            20
parsing   200         framing_iframe        firefox Ubuntu 22.04 121 selenium headless             18
basic     200         referrer_iframe       firefox Ubuntu 22.04 121 selenium headless             16
                                            firefox Ubuntu 22.04 122 selenium headless             16
          302         oac_window.open       edge Ubuntu 22.04 121 selenium headless-new            16
          200         referrer_iframe       edge Ubuntu 22.04 121 selenium headless-new            16
parsing   200         framing_iframe        edge Ubuntu 22.04 121 selenium headless-new            14
                      fullscreen_iframe     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     10
basic     302         oac_iframe            safari macOS 14.3 17.3 selenium real                   10
parsing   200         fullscreen_iframe     chrome Ubuntu 22.04 120 selenium headless-new          10
basic     302         referrer_iframe       edge Ubuntu 22.04 121 selenium headless-new             9
parsing   200         fullscreen_iframe     chrome Ubuntu 22.04 122 selenium headless-new           9
                      framing_iframe        brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      7
                                            chrome Ubuntu 22.04 122 selenium headless-new           7
basic     302         oac_window.open       safari macOS 14.3 17.3 selenium real                    6
parsing   200         fullscreen_iframe     firefox Ubuntu 22.04 122 selenium headless              5
                      framing_iframe        chrome Ubuntu 22.04 121 selenium headless-new           5
                      fullscreen_iframe     chrome Ubuntu 22.04 121 selenium headless-new           4
basic     200         referrer_iframe       chrome Ubuntu 22.04 120 selenium headless-new           4
parsing   200         upgradeHSTS_subdomain firefox Ubuntu 22.04 121 selenium headless              4
basic     302         referrer_iframe       firefox Ubuntu 22.04 122 selenium headless              3
                      oac_window.open       firefox Ubuntu 22.04 122 selenium headless              3
parsing   200         fetch_GET             firefox Ubuntu 22.04 121 selenium headless              3
basic     302         referrer_iframe       firefox Ubuntu 22.04 121 selenium headless              3
parsing   200         accesswindow_direct   firefox Ubuntu 22.04 122 selenium headless              2
basic     302         oac_iframe            firefox Ubuntu 22.04 122 selenium headless              2
                      oac_window.open       firefox Ubuntu 22.04 121 selenium headless              2
          200         referrer_iframe       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                            chrome Ubuntu 22.04 122 selenium headless-new           1
parsing   200         accesswindow_direct   edge Ubuntu 22.04 121 selenium headless-new             1
                                            chrome Ubuntu 22.04 121 selenium headless-new           1
                      imgloading_iframe     chrome Ubuntu 22.04 120 selenium headless-new           1
                

# General Test Statistics

In [9]:
df[["browser", "resp_type", "test_name", "relation_info", "response_id", "org_scheme", "org_host", "resp_scheme", "resp_host"]].drop_duplicates().groupby(["resp_type", "browser"]).count()

test_name  \
resp_type browser                                                         
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      10456   
          chrome Ubuntu 22.04 120 selenium headless-new           10456   
          chrome Ubuntu 22.04 121 selenium headless-new           10456   
          chrome Ubuntu 22.04 122 selenium headless-new           10456   
          edge Ubuntu 22.04 121 selenium headless-new             10456   
          firefox Ubuntu 22.04 121 selenium headless              10456   
          firefox Ubuntu 22.04 122 selenium headless              10456   
          safari macOS 14.3 17.3 selenium real                    10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     168774   
          chrome Ubuntu 22.04 120 selenium headless-new          168774   
          chrome Ubuntu 22.04 121 selenium headless-new          168774   
          chrome Ubuntu 22.04 122 selenium headless-new          168774   
          edge Ubuntu 22.04 121 selenium headless-new            168774   
          firefox Ubuntu 22.04 121 selenium headless             168774   
          firefox Ubuntu 22.04 122 selenium headless             168774   
          safari macOS 14.3 17.3 selenium real                   168774   

                                                              relation_info  \
resp_type browser                                                             
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...          10456   
          chrome Ubuntu 22.04 120 selenium headless-new               10456   
          chrome Ubuntu 22.04 121 selenium headless-new               10456   
          chrome Ubuntu 22.04 122 selenium headless-new               10456   
          edge Ubuntu 22.04 121 selenium headless-new                 10456   
          firefox Ubuntu 22.04 121 selenium headless                  10456   
          firefox Ubuntu 22.04 122 selenium headless                  10456   
          safari macOS 14.3 17.3 selenium real                        10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...         168774   
          chrome Ubuntu 22.04 120 selenium headless-new              168774   
          chrome Ubuntu 22.04 121 selenium headless-new              168774   
          chrome Ubuntu 22.04 122 selenium headless-new              168774   
          edge Ubuntu 22.04 121 selenium headless-new                168774   
          firefox Ubuntu 22.04 121 selenium headless                 168774   
          firefox Ubuntu 22.04 122 selenium headless                 168774   
          safari macOS 14.3 17.3 selenium real                       168774   

                                                              response_id  \
resp_type browser                                                           
basic     brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...        10456   
          chrome Ubuntu 22.04 120 selenium headless-new             10456   
          chrome Ubuntu 22.04 121 selenium headless-new             10456   
          chrome Ubuntu 22.04 122 selenium headless-new             10456   
          edge Ubuntu 22.04 121 selenium headless-new               10456   
          firefox Ubuntu 22.04 121 selenium headless                10456   
          firefox Ubuntu 22.04 122 selenium headless                10456   
          safari macOS 14.3 17.3 selenium real                      10456   
parsing   brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       168774   
          chrome Ubuntu 22.04 120 selenium headless-new            168774   
          chrome Ubuntu 22.04 121 selenium headless-new            168774   
          chrome Ubuntu 22.04 122 selenium headless-new            168774   
          edge Ubuntu 22.04 121 selenium headless-new              168774   
          firefox Ubuntu 22.04 121 selenium headless               168774   
          firefox Ubuntu 22.04 122 selenium headless    

In [10]:
# Some browsers only used the more response_ids per page mode, thus these numbers don't say much
df.groupby("browser")["clean_url"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    100726
chrome Ubuntu 22.04 120 selenium headless-new                           28713
chrome Ubuntu 22.04 121 selenium headless-new                          100726
chrome Ubuntu 22.04 122 selenium headless-new                           27889
edge Ubuntu 22.04 121 selenium headless-new                            100726
firefox Ubuntu 22.04 121 selenium headless                              29687
firefox Ubuntu 22.04 122 selenium headless                             100726
safari macOS 14.3 17.3 selenium real                                    85492
Name: clean_url, dtype: int64

In [11]:
# Number of response_ids tested (should be 44415)
# Many response_ids have more than one test (e.g., both iframe and window.open) thus this is not enough to show that all tests have results!
df.groupby("browser")["response_id"].nunique()

browser
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    44415
chrome Ubuntu 22.04 120 selenium headless-new                          44415
chrome Ubuntu 22.04 121 selenium headless-new                          44415
chrome Ubuntu 22.04 122 selenium headless-new                          44415
edge Ubuntu 22.04 121 selenium headless-new                            44415
firefox Ubuntu 22.04 121 selenium headless                             44415
firefox Ubuntu 22.04 122 selenium headless                             44415
safari macOS 14.3 17.3 selenium real                                   44415
Name: response_id, dtype: int64

In [12]:
# Make sure that all responses actually have responses
# (create_repeat.py assumes that at least one browser collected the results)

r1 = df.groupby("label")["response_id"].nunique().sort_values()
r2 = responses.loc[responses["resp_type"] != "debug"].groupby("label")["id"].nunique().sort_values()

pd.concat([r1, r2], axis=1)

,response_id,id
label,,
CORS,6,6
CORS-ACAC,864,864
CSPvsXFO,1447,1447
CORS-ACAM,1576,1576
CORS-ACEH,1577,1577
CORS-ACAH,1577,1577
TAO,1614,1614
OAC,1713,1713
CORS-ACAO,2465,2465


In [13]:
# Number of response_ids for each "group": resp_type, test_name, relation_info
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14
parsing   accesswindow_direct           direct                 2656
          fetch_GET                     credentials            8059
                                        custom_headers         8059
                                        simple                 8059
          fetch_TEST                    custom_method          8059
          framing_iframe                direct                 8780
                                        nested                 8780
          fullscreen_iframe             child_allow            3880
                                        direct                 3880
          imgloading_iframe             direct                 2710
          oac_window.open               window.open            1699
          perfAPI_img                   direct                 1601
          referrer_iframe               iframe                 2797
          script_execution_iframe       direct                 2726
          subresourceloadingCOEP_img    direct                 2637
          subresourceloadingCORP_img    direct                 3357
          upgradeHSTS_direct            direct                 3324
          upgradeHSTS_subdomain         subdomain              3324

# Stability (Results after 2x with only one rseponse_id per parsing URL test)
- Basic tests:
    - 21/(10456x5) tests with different outcomes!
    - 13x firefox subresourceloadingCOEP_img (random behavior bug)
    - 6x timeout vs message 4(x edge, 2x brave)
    - 1x brave referrer different outcome
    - 1x safari COEP different outcome?
- Parsing tests:
    - 755/(168774x5) tests with different outcomes!
    - quite a lot! last time we only had 1 single test with a different outcome? (We used different versions of the browsers)
    - 3x accesswindow_direct (null vs accessible), 1x fullscreen_iframe (timeout vs message), 1x img_loading_iframe (timeout vs message)
    - All others (700+) upgradeHSTS_subdomain and upgradeHSTS_direct
        - Prevalence: `brave ~= edge > chrome >> firefox >> safari`
        - Hints at systematic issue: maybe the reset did not work or something similar?
        - Mostly timeout vs redirect=false; however some are also timeout vs redirect=false vs redirect=true!
        - Timeout maybe related to the HTTPS upgrade feature in chromium-based browsers which can take some time (>5s?)? (Non-Authoritative-Reason: HttpsUpgrades)
        - Unclear how redirect true vs false could happen? Maybe HSTS clearing did not work?
        - Manual testing always results in redirect=false for all tried clean_urls!

### Update: multiple response_ids per parsing URL test
- basic (123+)
    - mostly firefox subresourceloadingCOEP_img (random behavior bug)
    - some rare instances of other tests (timeouts)
- parsing (5000+)
    - fetch_* timeout issue in firefox (fixed)
    - subresourceloadingCOEP_img and imgloading_iframe timeout issues (other browsers) (probably fixed)
    - upgradeHSTS_subdomain + upgradeHSTS_direct
    - rare instances of other tests
- mostly timeouts?: Firefox fetch, other browsers subresourceloadingCOEP_img
- maybe we have too many tests per page/or the timeout is not high enough -> decreased the number of tests for some feature groups (now it seems better)
- (OT: if running `desktop_selenium.py` in the repeat mode, one has to increate `--timeout_task` (15*100 = 1500s > 1000) or decrease `--max_urls_until_restart` as we ran the repeat tests with a 3xTIMEOUT)

In [14]:
# Remove all tests that timed out (test_status == 0), each test has at least one result that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [15]:
for group_name, resp_type_group in df.groupby("resp_type"):
    #if group_name == "basic":
    #    continue
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])
    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()
    
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show the first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

basic
123 tests have different outcomes!


,count
browser,
firefox Ubuntu 22.04 122 selenium headless,54
firefox Ubuntu 22.04 121 selenium headless,53
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,6
chrome Ubuntu 22.04 122 selenium headless-new,3
chrome Ubuntu 22.04 120 selenium headless-new,3
safari macOS 14.3 17.3 selenium real,2
chrome Ubuntu 22.04 121 selenium headless-new,1
edge Ubuntu 22.04 121 selenium headless-new,1


count
(test_name, )              (browser, )                                              
subresourceloadingCOEP_img firefox Ubuntu 22.04 122 selenium headless             54
                           firefox Ubuntu 22.04 121 selenium headless             52
referrer_iframe            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      6
accesswindow_direct        safari macOS 14.3 17.3 selenium real                    2
referrer_iframe            chrome Ubuntu 22.04 120 selenium headless-new           2
framing_embed              firefox Ubuntu 22.04 121 selenium headless              1
oac_window.open            chrome Ubuntu 22.04 122 selenium headless-new           1
referrer_iframe            chrome Ubuntu 22.04 121 selenium headless-new           1
subresourceloadingCOEP_img chrome Ubuntu 22.04 120 selenium headless-new           1
referrer_iframe            chrome Ubuntu 22.04 122 selenium headless-new           1
subresourceloadingCOEP_img chrome Ubuntu 22.04 122 selenium headless-new           1
upgradeHSTS_direct         edge Ubuntu 22.04 121 selenium headless-new             1

accesswindow_direct [81]


,,count
outcome_str,browser,
{'window.open.opener': 'null'},safari macOS 14.3 17.3 selenium real,3
"{'window.open.opener': 'object ""[object Window]""'}",safari macOS 14.3 17.3 selenium real,1


...

,,count
outcome_str,browser,
{'response.redirected': False},firefox Ubuntu 22.04 122 selenium headless,240
Fetch timed out,firefox Ubuntu 22.04 122 selenium headless,24


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
2024211,firefox Ubuntu 22.04 122 selenium headless,{'response.redirected': False},"[['strict-transport-security', 'max-age=60(']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32174&last_id=32174&scheme=http
2064533,firefox Ubuntu 22.04 122 selenium headless,Fetch timed out,"[['strict-transport-security', 'max-a?ge=60']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=32149&last_id=32149&scheme=http


upgradeHSTS_subdomain [29963, 33015]


,,count
outcome_str,browser,
Fetch timed out,safari macOS 14.3 17.3 selenium real,2
{'response.redirected': False},safari macOS 14.3 17.3 selenium real,2


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
2095593,safari macOS 14.3 17.3 selenium real,{'response.redirected': False},"[['strict-transport-securityO', 'max-age=0']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=29963&last_id=29963&scheme=http
2104878,safari macOS 14.3 17.3 selenium real,Fetch timed out,"[['strict-transport-securityO', 'max-age=0']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=parsing&browser_id=1&label=HSTS&first_id=29963&last_id=29963&scheme=http


## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [34]:
# How many outcomes did we collect for each test?!
# Should be between 5 and 20+ (less than 5 is concerning as we cannot do proper majority voting in such cases)

final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    res = d.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()
    counts = counts.rename(columns={"count": group})

    only_once = res.loc[res["count"] == 1]
    less_than_five = res.loc[res["count"] < 5]
    if 1 < len(only_once) < 200:
        print(f"Tests with only one collected outcome: {group}")
        display(only_once.reset_index()[["test_name", "relation_info"]].value_counts().to_frame())

    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

safari macOS 14.3 17.3 selenium real


,,count
test_name,relation_info,
accesswindow_direct,direct,13
referrer_iframe,iframe,9
perfAPI_img,direct,8
referrer_iframe,window.open,6


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
count,,,,,,,,
11,168588.0,NaN,96483.0,NaN,163427.0,NaN,166350.0,NaN
24,10455.0,10452.0,10456.0,7608.0,10404.0,10443.0,10437.0,NaN
10,184.0,5.0,2.0,NaN,5347.0,8.0,2420.0,NaN
9,2.0,168762.0,NaN,165426.0,NaN,150438.0,NaN,NaN
23,1.0,4.0,NaN,2847.0,43.0,10.0,14.0,NaN
8,NaN,7.0,NaN,2978.0,NaN,18325.0,NaN,NaN
12,NaN,NaN,72289.0,NaN,NaN,3.0,NaN,NaN
7,NaN,NaN,NaN,370.0,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,1.0,9.0,2.0,5.0,NaN


In [35]:
# Which tests have how many outcomes?

grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 256):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,256.000000,256.000000,256.000000
mean,16.088718,5600.925781,55452.757812
std,8.106265,10301.094819,98930.395846
min,1.997502,28.000000,112.000000
25%,9.000000,284.000000,6269.000000
50%,11.500000,1680.000000,28818.000000
75%,24.000000,5487.500000,58706.250000
max,24.000000,48354.000000,556071.000000


mean  \
browser                                            resp_type test_name                                  
firefox Ubuntu 22.04 122 selenium headless         basic     upgradeHSTS_direct             24.000000   
                                                             upgradeHSTS_subdomain          24.000000   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     oac_iframe                     24.000000   
                                                             oac_window.open                24.000000   
                                                             perfAPI_img                    24.000000   
                                                             script_execution_iframe        24.000000   
                                                             subresourceloadingCOEP_img     24.000000   
                                                             subresourceloadingCORP_img     24.000000   
                                                             subresourceloadingCORP_object  24.000000   
                                                             upgradeHSTS_direct             24.000000   
                                                             upgradeHSTS_subdomain          24.000000   
firefox Ubuntu 22.04 122 selenium headless         basic     perfAPI_img                    24.000000   
                                                             script_execution_iframe        24.000000   
                                                             subresourceloadingCOEP_img     24.000000   
                                                             subresourceloadingCORP_img     24.000000   
                                                             subresourceloadingCORP_object  24.000000   
                                                             accesswindow_direct            24.000000   
                                                             fetch_GET                      24.000000   
                                                             fetch_TEST                     24.000000   
                                                             framing_embed                  24.000000   
                                                             framing_iframe                 24.000000   
                                                             framing_object                 24.000000   
                                                             fullscreen_iframe              24.000000   
                                                             imgloading_iframe              24.000000   
firefox Ubuntu 22.04 121 selenium headless         basic     upgradeHSTS_direct             24.000000   
                                                             upgradeHSTS_subdomain          24.000000   
chrome Ubuntu 22.04 120 selenium headless-new      basic     accesswindow_direct            24.000000   
                                                             fetch_GET                      24.000000   
                                                             fetch_TEST                     24.000000   
                                                             framing_embed                  24.000000   
                                                             framing_iframe                 24.000000   
                                                             framing_object                 24.000000   
                                                             fullscreen_iframe              24.000000   
                                                             imgloading_iframe              24.000000   
                                                             oac_iframe                     24.000000   
                                                             oac_window.open                24.000000   
                                                             perfAPI_img                    24.000000   
  

In [36]:
# Different outcomes (in percentage for a test group (test_name)
# (Does not take into account how often each test was executed)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 67):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,256.000000,256.000000,256.000000
mean,0.004021,5600.925781,22.007812
std,0.016598,10301.094819,94.867916
min,0.000000,28.000000,0.000000
25%,0.000000,284.000000,0.000000
50%,0.000000,1680.000000,0.000000
75%,0.000093,5487.500000,1.000000
max,0.153409,48354.000000,979.000000


,,,mean,count,sum
browser,resp_type,test_name,,,
firefox Ubuntu 22.04 122 selenium headless,basic,subresourceloadingCOEP_img,0.153409,352,54
firefox Ubuntu 22.04 121 selenium headless,basic,subresourceloadingCOEP_img,0.147727,352,52
chrome Ubuntu 22.04 122 selenium headless-new,parsing,subresourceloadingCOEP_img,0.101631,5274,536
chrome Ubuntu 22.04 121 selenium headless-new,parsing,subresourceloadingCOEP_img,0.052522,5274,277
chrome Ubuntu 22.04 122 selenium headless-new,parsing,imgloading_iframe,0.051107,5420,277
edge Ubuntu 22.04 121 selenium headless-new,parsing,subresourceloadingCOEP_img,0.045127,5274,238
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,parsing,upgradeHSTS_subdomain,0.044826,6648,298
edge Ubuntu 22.04 121 selenium headless-new,parsing,upgradeHSTS_subdomain,0.044073,6648,293
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,parsing,upgradeHSTS_direct,0.037154,6648,247


In [19]:
# Run the test groups with many different outcomes again?
# Groups with non-negligble different outcomes are subresourceloadingCOEP_img, upgradeHSTS_subdomain, upgradeHSTS_direct, imgloading_iframe, fetch_TEST, fetch_GET
test_files_to_repeat =  ["fetch-cors.sub.html", "subresource-loading-coep.sub.html", "subresource-loading-csp.sub.html", "upgrade-hsts.sub.html"]

# TODO: implement utils.py/desktop_selenium mode to only rerun these files?!
# For now, simply run everything again to get some more results for stability?!

In [20]:
# Majority voting for the tests with multiple outcomes
df.sample(10000).groupby(["test_name", "relation_info", "browser", "org_host", "resp_scheme", "org_origin", "resp_origin", "response_id"])["outcome_str"].agg([pd.Series.mode, "nunique", "count"]).reset_index().sort_values("count")

,test_name,relation_info,browser,org_host,resp_scheme,org_origin,resp_origin,response_id,mode,nunique,count
6638,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,36524,fullscreenEnabled: true,1,1
6629,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,35854,message timeout,1,1
6630,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,35958,fullscreenEnabled: true,1,1
6631,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,35970,fullscreenEnabled: true,1,1
6632,fullscreen_iframe,direct,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,35989,fullscreenEnabled: true,1,1
...,...,...,...,...,...,...,...,...,...,...,...
5678,framing_iframe,nested,firefox Ubuntu 22.04 122 selenium headless,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,2132,message send,1,2
6773,fullscreen_iframe,direct,chrome Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.webappsec.eu,36025,fullscreenEnabled: true,1,2
3660,framing_embed,direct,chrome Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://sub.headers.websec.saarland,30,message send,1,2
8755,subresourceloadingCOEP_img,direct,firefox Ubuntu 22.04 122 selenium headless,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://headers.webappsec.eu,19565,"{'image-events': {'swag.jpg': 'load', 'swag-sa...",1,2


# Browser differences!
- First perform majority voting
- Then diff the browsers

In [33]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [22]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs, >= 5)

print("Original data entries", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)
df = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type", "label", "name"])["outcome_str"].agg(pd.Series.mode).reset_index()
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 14195906
Only one row for each test (per browser; majority voting) 1433837


In [23]:
# If several values occur the same, the mode is an np.ndarray and not a string!
# What to do in such cases? If we have more repetitions the issue should probably go away automatically?
print("Tests with the same mode (highest frequently outcome):")
display(df.loc[df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray))].groupby(["browser", "resp_type", "test_name"])["outcome_str"].count())

# In the rare cases, where it actually matters, we can just take the first? (this might be noise in the generated trees)
df['outcome_str'] = df['outcome_str'].apply(lambda x: x[0] if isinstance(x, np.ndarray) else x)

Tests with the same mode (highest frequently outcome):


browser                               resp_type  test_name            
safari macOS 14.3 17.3 selenium real  parsing    imgloading_iframe        1
                                                 upgradeHSTS_direct       3
                                                 upgradeHSTS_subdomain    2
Name: outcome_str, dtype: int64

In [24]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,171048
2,8124
3,58


In [25]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# couple have 0%

# Parsing:
# oac_window.open has 99%
# Other test groups have between 0.5 - 7.5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count   sum      mean
resp_type test_name                     relation_info                        
basic     subresourceloadingCORP_object direct            176   176  1.000000
parsing   oac_window.open               window.open      3398  3370  0.991760
basic     oac_window.open               window.open       224   192  0.857143
          oac_iframe                    direct            224   144  0.642857
                                        sandbox           224   144  0.642857
          referrer_iframe               window.open       256   117  0.457031
          fullscreen_iframe             direct            384   142  0.369792
          framing_embed                 direct            560   186  0.332143
          framing_object                direct            560   186  0.332143
          fullscreen_iframe             child_allow       384   113  0.294271
          subresourceloadingCOEP_img    direct            176    49  0.278409
          perfAPI_img                   direct            208    55  0.264423
          framing_object                sandbox           560   120  0.214286
                                        nested            560   120  0.214286
          framing_embed                 nested            560   120  0.214286
                                        sandbox           560   120  0.214286
          accesswindow_direct           direct            176    28  0.159091
          script_execution_iframe       sandbox           272    21  0.077206
          referrer_iframe               iframe            256    18  0.070312
          fullscreen_iframe             child             384    23  0.059896
          imgloading_iframe             direct            272    13  0.047794
parsing   upgradeHSTS_direct            direct           6648   307  0.046179
          fullscreen_iframe             child_allow      7760   355  0.045747
          imgloading_iframe             direct           5420   230  0.042435
          script_execution_iframe       direct           5452   216  0.039618
basic     upgradeHSTS_direct            direct             28     1  0.035714
parsing   fullscreen_iframe             direct           7760   272  0.035052
basic     subresourceloadingCOEP_img    sandbox           176     4  0.022727
parsing   framing_iframe                direct          17560   365  0.020786
          referrer_iframe               iframe           5594   101  0.018055
          subresourceloadingCOEP_img    direct           5274    85  0.016117
          upgradeHSTS_subdomain         subdomain        6648   103  0.015493
basic     framing_iframe                sandbox           560     8  0.014286
parsing   perfAPI_img                   direct           3202    33  0.010306
          subresourceloadingCORP_img    direct           6714    68  0.010128
          framing_iframe                nested          17560   144  0.008200
          accesswindow_direct           direct           5312    40  0.007530
          fetch_GET                     simple          16118   104  0.006452
                                        custom_headers  16118   101  0.006266
          fetch_TEST                    custom_method   16118   101  0.006266
basic     subresourceloadingCORP_img    direct            176     1  0.005682
parsing   fetch_GET                     credentials     16118    86  0.005336
basic     fetch_GET                     credentials        96     0  0.000000
                                        simple             96     0  0.000000
                                        custom_headers     96     0  0.000000
          fullscreen_iframe             child_sandbox     384     0  0.000000
          framing_iframe                nested            560     0  0.000000
                                        direct            560     0  0.000000
          fetch_TEST                    custom_method      96     0  0.000000
          upgradeHSTS_subdomain         subdomain          28     0  0.000000


In [26]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]
tests_with_more_than_one_outcome = df_mult.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])

all_tests_should_be = (168774+10456) * 8 # (num parsing tests + num_basic_tests) * num_browsers
print(f"All test rows: {len(df)}, should be: {all_tests_should_be}, Rows with more than one outcome: {len(df_mult)}, Tests with more than one outcome: {len(tests_with_more_than_one_outcome)}")

All test rows: 1433837, should be: 1433840, Rows with more than one outcome: 65456, Tests with more than one outcome: 8182


In [27]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
4


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
"{'window.open.opener': 'object ""[object Window]""'}",[safari macOS 14.3 17.3 selenium real]


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
"{'window.open.opener': 'object ""[object Window]""'}","[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"


...

,browser
outcome_str,
{'response.redirected': False},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"
{'response.redirected': True},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"


,browser
outcome_str,
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
{'response.redirected': True},[safari macOS 14.3 17.3 selenium real]


,browser
outcome_str,
TypeError: Failed to fetch,"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
{'response.redirected': False},[safari macOS 14.3 17.3 selenium real]


,browser
outcome_str,
Fetch timed out,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless]"
{'response.redirected': True},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
{'response.redirected': True},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, safari macOS 14.3 17.3 selenium real]"


In [28]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['browser'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(group)
    unique_outcomes = {}
    group.groupby(["browser"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

,['safari macOS 14.3 17.3 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
{'window.open.opener': 'null'},16,24,56
"{'window.open.opener': 'object ""[object Window]""'}",52,44,12


,"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['safari macOS 14.3 17.3 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']"
"{'error': 'null', 'headers': 'content-length,'}",3.0,9.0,10
"{'error': 'null', 'headers': 'content-length,test,'}",19.0,31.0,20
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",64.0,46.0,16
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,40


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'safari macOS 14.3 17.3 selenium real']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['safari macOS 14.3 17.3 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'safari macOS 14.3 17.3 selenium real']"
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",2.0,4.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",14.0,NaN,4.0,10.0,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",24.0,NaN,25.0,34.0,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",21.0,NaN,68.0,13.0,NaN
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,NaN,NaN,40.0


,"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'safari macOS 14.3 17.3 selenium real']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['safari macOS 14.3 17.3 selenium real']
"{'error': 'null', 'headers': ''}",3.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,location,server,test,'}",1.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",4.0,2.0,NaN,NaN
"{'error': 'null', 'headers': 'cache-control,content-type,'}",NaN,10.0,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",NaN,14.0,4.0,10.0
"{'error': 'null', 'headers': 'content-length,test,'}",NaN,26.0,25.0,40.0
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,40.0,NaN,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",NaN,12.0,67.0,46.0


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'safari macOS 14.3 17.3 selenium real']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['safari macOS 14.3 17.3 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'safari macOS 14.3 17.3 selenium real']"
"{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-origin,access-control-expose-headers,content-length,date,server,test,'}",2.0,4.0,NaN,NaN,NaN
"{'error': 'null', 'headers': 'content-length,'}",14.0,NaN,4.0,10.0,NaN
"{'error': 'null', 'headers': 'content-length,test,'}",24.0,NaN,25.0,34.0,NaN
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",21.0,NaN,68.0,13.0,NaN
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,NaN,NaN,NaN,40.0


...

,['safari macOS 14.3 17.3 selenium real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']"
"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",4.0,NaN
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",NaN,4.0


,['safari macOS 14.3 17.3 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'safari macOS 14.3 17.3 selenium real']"
error,17.0,29.0,52.0,NaN
timeout: no event fired,12.0,NaN,NaN,NaN
load,NaN,NaN,17.0,40.0


,"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']",['safari macOS 14.3 17.3 selenium real']
error,44.0,76.0,NaN
load,132.0,100.0,NaN
timeout: no event fired,NaN,NaN,176.0


,['safari macOS 14.3 17.3 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
Fetch timed out,3.0,12,NaN
{'response.redirected': False},39.0,281,69.0
{'response.redirected': True},266.0,6,212.0
TypeError: Failed to fetch,NaN,9,27.0


,['safari macOS 14.3 17.3 selenium real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
Fetch timed out,2.0,12,NaN
{'response.redirected': False},42.0,79,40.0
{'response.redirected': True},59.0,3,36.0
TypeError: Failed to fetch,NaN,9,27.0


In [29]:
# Simmilarity between browsers and versions!

# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    if show_all:
        display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,Absolute Diff,Percentage Diff,Count
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,0.000000,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_chrome Ubuntu 22.04 120 selenium headless-new,141,0.078670,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_chrome Ubuntu 22.04 121 selenium headless-new,142,0.079228,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_chrome Ubuntu 22.04 122 selenium headless-new,142,0.079228,179230
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new_vs_edge Ubuntu 22.04 121 selenium headless-new,142,0.079228,179230
...,...,...,...
safari macOS 14.3 17.3 selenium real_vs_chrome Ubuntu 22.04 122 selenium headless-new,6644,3.706969,179230
safari macOS 14.3 17.3 selenium real_vs_edge Ubuntu 22.04 121 selenium headless-new,6644,3.706969,179230
safari macOS 14.3 17.3 selenium real_vs_firefox Ubuntu 22.04 121 selenium headless,2754,1.536573,179230
safari macOS 14.3 17.3 selenium real_vs_firefox Ubuntu 22.04 122 selenium headless,2740,1.528762,179230


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,141,142,142,142,6900,6886,6602
chrome Ubuntu 22.04 120 selenium headless-new,141,0,1,1,1,6927,6913,6643
chrome Ubuntu 22.04 121 selenium headless-new,142,1,0,0,0,6928,6914,6644
chrome Ubuntu 22.04 122 selenium headless-new,142,1,0,0,0,6928,6914,6644
edge Ubuntu 22.04 121 selenium headless-new,142,1,0,0,0,6928,6914,6644
firefox Ubuntu 22.04 121 selenium headless,6900,6927,6928,6928,6928,0,16,2754
firefox Ubuntu 22.04 122 selenium headless,6886,6913,6914,6914,6914,16,0,2740
safari macOS 14.3 17.3 selenium real,6602,6643,6644,6644,6644,2754,2740,0


accesswindow_direct


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,56,40
chrome Ubuntu 22.04 120 selenium headless-new,0,56,40
chrome Ubuntu 22.04 121 selenium headless-new,0,56,40
chrome Ubuntu 22.04 122 selenium headless-new,0,56,40
edge Ubuntu 22.04 121 selenium headless-new,0,56,40
firefox Ubuntu 22.04 121 selenium headless,56,0,40
firefox Ubuntu 22.04 122 selenium headless,56,0,40
safari macOS 14.3 17.3 selenium real,40,40,0


...

,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,32,176
chrome Ubuntu 22.04 120 selenium headless-new,0,32,176
chrome Ubuntu 22.04 121 selenium headless-new,0,32,176
chrome Ubuntu 22.04 122 selenium headless-new,0,32,176
edge Ubuntu 22.04 121 selenium headless-new,0,32,176
firefox Ubuntu 22.04 121 selenium headless,32,0,176
firefox Ubuntu 22.04 122 selenium headless,32,0,176
safari macOS 14.3 17.3 selenium real,176,176,0


upgradeHSTS_direct


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,244,87
chrome Ubuntu 22.04 120 selenium headless-new,0,244,87
chrome Ubuntu 22.04 121 selenium headless-new,0,244,87
chrome Ubuntu 22.04 122 selenium headless-new,0,244,87
edge Ubuntu 22.04 121 selenium headless-new,0,244,87
firefox Ubuntu 22.04 121 selenium headless,244,0,285
firefox Ubuntu 22.04 122 selenium headless,244,0,285
safari macOS 14.3 17.3 selenium real,87,285,0


upgradeHSTS_subdomain


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,safari macOS 14.3 17.3 selenium real
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,68,56
chrome Ubuntu 22.04 120 selenium headless-new,0,68,56
chrome Ubuntu 22.04 121 selenium headless-new,0,68,56
chrome Ubuntu 22.04 122 selenium headless-new,0,68,56
edge Ubuntu 22.04 121 selenium headless-new,0,68,56
firefox Ubuntu 22.04 121 selenium headless,68,0,82
firefox Ubuntu 22.04 122 selenium headless,68,0,82
safari macOS 14.3 17.3 selenium real,56,82,0


In [30]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

test_name                   label     relation_info 
framing_iframe              XFO       direct            74944
                                      nested            74944
fullscreen_iframe           PP        child_allow       65152
                                      direct            65152
subresourceloadingCORP_img  CORP      direct            55120
                                                        ...  
framing_object              CSPvsXFO  sandbox             896
fetch_GET                   CORS      simple              768
                                      credentials         768
                                      custom_headers      768
fetch_TEST                  CORS      custom_method       768
Name: count, Length: 73, dtype: int64

In [31]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,22 hours 30 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 15 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,100 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Remove tests that are the same in all browsers 65456
('accesswindow_direct', 'direct')


KeyError: "['org_origin', 'resp_origin', 'status_code', 'raw_header'] not in index"

# Manual analysis

In [ ]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()